# Coal mining disaster

This is similar to https://docs.pymc.io/projects/examples/en/latest/BART/BART_introduction.html

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm

In [ ]:
RANDOM_SEED = 8457
rng = np.random.RandomState(RANDOM_SEED)
az.style.use("arviz-darkgrid")

In [ ]:
np.loadtxt("coal.csv")

In [ ]:
# discretize data
years = int(coal.max() - coal.min())
bins = years // 4
hist, x_edges = np.histogram(coal, bins=bins)
# compute the location of the centers of the discretized data
x_centers = x_edges[:-1] + (x_edges[1] - x_edges[0]) / 2
# xdata needs to be 2D for BART
x_data = x_centers[:, None]
# express data as the rate number of disaster per year
y_data = hist / 4

In [ ]:
with pm.Model(rng_seeder=rng) as model_coal:
    μ = pm.BART("μ", X=x_data, Y=y_data, m=20)
    y_pred = pm.Poisson("y_pred", mu=pm.math.exp(μ), observed=y_data)
    idata_coal = pm.sample()

In [ ]:
_, ax = plt.subplots(figsize=(10, 6))

rates = np.exp(idata_coal.posterior["μ"])
rate_median = np.exp(idata_coal.posterior["μ"].median(dim=["draw", "chain"]))
ax.plot(x_centers, rate_median, "w", lw=3)
az.plot_hdi(x_centers, rates, smooth=False)
az.plot_hdi(x_centers, rates, hdi_prob=0.5, smooth=False, plot_kwargs={"alpha": 0})
ax.plot(coal, np.zeros_like(coal) - 0.5, "k|")
ax.set_xlabel("years")
ax.set_ylabel("rate");